In [1]:
import pytest
import ipytest

ipytest.config(rewrite_asserts=True, magics=True)

__file__ = 'exercises-geometry.ipynb' 

In [138]:
import math

class Point:
    def __init__(self, x,y=None):
        self.x =float(x)
        self.y =float(y)
    
    def normal(self):
        return Point(-self.y, self.x)
    
    def normalize(self, d=0):
        if d == 0:
            d = math.sqrt(self.x*self.x +self.y*self.y)
        return Point(self.x/d, self.y/d)
    
    def scalar_product(self,other: 'Point'):
        return self.x*other.x + self.y*other.y
    
    def is_orthogonal(self, other: 'Point'):
        product =self.scalar_product(other)
        return math.isclose(product,0)
    
    def is_collinear(self, other: 'Point'):
        return self.is_orthogonal(other.normal())
    
    @staticmethod
    def distance(a:'Point', b: 'Point') -> float:
        dx = a.x-b.x
        dy = a.y-b.y
        return math.sqrt(dx*dx + dy*dy)
    
    def __add__(self, other: 'Point'):
        r = NotImplemented
        if isinstance(other, Point):
            r = Point(self.x+other.x, self.y+other.y)
        return r
    
    def __mul__(self,other):
        r = NotImplemented
        if isinstance(other, float) or isinstance(other, int):
            factor = float(other)
            r = Point(self.x*factor, self.y*factor)
        return r

    def __eq__(self, other: 'Point'):
        r = NotImplemented
        if isinstance(other, Point):
            r = math.isclose(self.x,other.x,abs_tol=1e-9) and  math.isclose(self.y,other.y,abs_tol=1e-9)
        return r
    
    def __repr__(self):
        return f'({self.x}, {self.y})'
    
        
class Segment:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        
class Line:
    def __init__(self, point: Point, vector: Point):
        self.point = point
        self.vector = vector
        
    def contains(self, point: Point):
        a = self.point
        b = point
        v_ab = Point(a.x -b.x, a.y - b.y)
        return v_ab.is_collinear(self.vector)
        
        
class CircularArc:
    def __init__(self, start: Point, end: Point, radius: float):
        self.start = start
        self.end = end
        self.radius = radius
        self.angle = 2 * math.asin(Point.distance(start, end) / (2* radius))
        self.length = self.angle * math.pi * self.radius
        self.center = CircularArc.compute_center(start, end, radius, angle)
    
    @staticmethod
    def compute_center(start, end, radius, angle):
        chord = Point((end.x - start.x), (end.y - start.y))
        c = Point((end.x + start.x)/2, (end.y + start.y)/2)
        ortho_c = chord.normal()
        ortho_c = ortho_c.normalize()
        center = ortho_c*radius*math.cos(angle/2) + c
        return center
        
    @staticmethod
    def find_circular_arc_for(start: Point, end: Point, tangent: Point):
        angle, _, distance = CircularArc.find_angle_and_chord_vector(start, end, tangent)
        radius = distance /(2 * math.sin(angle/2))
        return CircularArc(start,end,radius)
    
    @staticmethod
    def find_angle_and_chord_vector(start, end, tangent):
        a = start
        b = end
        v = tangent
        
        distance = Point.distance(a, b)
        u = Point(b.x - a.x, b.y - a.y)
        u = u.normalize(distance)
        v = v.normalize()
        angle = math.acos(u.scalar_product(v))
        sign = u.x*v.y + u.y*v.x
        angle = 2 * angle * sign
        
        return angle, u, distance

In [136]:
a = 6.123233995736766e-17
math.isclose(0.,a,abs_tol=1e-9)


True

In [142]:
%%run_pytest[clean] -qq
import pytest

def test_add():
    a = Point(0.5, 0.5)
    b = Point(-1, 1)
    c = a + b
    assert c.x == -0.5
    assert c.y == 1.5
    
def test_equal():
    a = Point(0.5, 0.5)
    b = Point(0.5*(1+1e-10), 0.5)
    assert a == b
    
    
def test_equal_10e17():
    a = Point(0.0, 6.123233995736766e-17)
    b = Point(0.,0.)
    assert a == b

def test_are_orthogonal():
    a = Point(0.5, 0.5)
    b = Point(-1, 1)
    assert a.is_orthogonal(b)
    
def test_are_not_orthogonal():
    a = Point(1, 0.7)
    b = Point(0.5, 1)
    assert not a.is_orthogonal(b)
    
def test_are_collinear():
    a = Point(0.5, 0.5)
    b = Point(1, 1)
    assert a.is_collinear(b)
    

def test_coordinate_belongs_to_line():
    A = Point(1,2)
    line = Line(A, Point(1,1))
    assert line.contains( Point(2,3))
    
def test_distance_a_b():
    assert math.isclose(Point.distance(Point(1,0), Point(0,1)), math.sqrt(2))
    
def test_circular_arc():
    arc = CircularArc(start=Point(1,0), end=Point(0,1), radius=1.0)
    assert math.isclose(arc.angle,(math.pi/2.0))
    

table_vectors_angles = [
    ((-1,0), (1,0), math.pi),
    ((1,0), (0,1), math.pi/2),
    ((1,0), (0,-1), math.pi/2)

]
@pytest.mark.parametrize('a, b, angle', table_vectors_angles)
def test_find_angle_from_2_vectors(a, b, angle):
    def f(u,v):
        return math.acos(u.scalar_product(v))
        
    assert f(Point(a[0],a[1]), Point(b[0],b[1])) == angle
    

def test_find_find_angle_and_chord_vector():
    angle, u, distance = CircularArc.find_angle_and_chord_vector(Point(-1,0), 
                                                 Point(1,0), 
                                                 Point(0,1))    
    assert angle == math.pi
    
def test_find_circular_arc_for_simple_arc():
    candidat = CircularArc.find_circular_arc_for(Point(-1,0), 
                                                 Point(1,0), 
                                                 Point(0,1))
    assert candidat.radius == 1
    assert candidat.angle == math.pi
    assert candidat.center == Point(0.,0.)
    
def test_find_circular_arc_for_45deg_arc():
    candidat = CircularArc.find_circular_arc_for(Point(1,0), 
                                                 Point(0,1), 
                                                 Point(0,1))
    #assert math.isclose(candidat.angle, math.pi/2)
    assert candidat.center == (0,0)
    assert candidat.radius == 1    
    
def test_find_circular_arc_for_reverse_45deg_arc():
    candidat = CircularArc.find_circular_arc_for(Point(1,0), 
                                                 Point(0,1), 
                                                 Point(0,-1))
    assert candidat.radius == 1    
    assert math.isclose(candidat.angle, math.pi/2)    


..............FF                                                                                                                                                                                            [100%]
==================================================================================================== FAILURES =====================================================================================================
______________________________________________________________________________________ test_find_circular_arc_for_45deg_arc _______________________________________________________________________________________

    def test_find_circular_arc_for_45deg_arc():
        candidat = CircularArc.find_circular_arc_for(Point(1,0),
                                                     Point(0,1),
                                                     Point(0,1))
        #assert math.isclose(candidat.angle, math.pi/2)
>       assert candidat.center == (0,0)
E       assert (0.5000000